<a href="https://colab.research.google.com/github/psahai10/AIFellowship/blob/main/Copy_of_Zeroshot_Tasteatlas_(27150_images_289_classes).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparing Colab

In [ ]:
import subprocess
 
CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)
 
if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

CUDA version: 11.0


In [ ]:
!pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} -f https://download.pytorch.org/whl/torch_stable.html
!pip install ftfy regex

import torch

print("Torch version:", torch.__version__)

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████         | 834.1MB 1.3MB/s eta 0:04:18tcmalloc: large alloc 1147494400 bytes == 0x556c6718c000 @  0x7fe47ea83615 0x556c2e69a06c 0x556c2e779eba 0x556c2e69ce8d 0x556c2e78e99d 0x556c2e710fe9 0x556c2e70bb0e 0x556c2e69e77a 0x556c2e710e50 0x556c2e70bb0e 0x556c2e69e77a 0x556c2e70d86a 0x556c2e78f7c6 0x556c2e70cee2 0x556c2e78f7c6 0x556c2e70cee2 0x556c2e78f7c6 0x556c2e70cee2 0x556c2e78f7c6 0x556c2e811431 0x556c2e772049 0x556c2e6dcc84 0x556c2e69d8e9 0x556c2e711ade 0x556c2e69e69a 0x556c2e70ca45 0x556c2e70be0d 0x556c2e69e77a 0x556c2e70ca45 0x556c2e69e69a 0x556c2e70ca45
     |█████████████████████████████▏  | 1055.7MB 1.2MB/s eta 0:01:27tcmalloc: large alloc 1434370048 bytes == 0x556cab7e2000 @  0x7fe47ea83615 0x556c2e69a06c 0x556c2e779eba 0x556c2e69ce8d 0x556c2e78e99d 0x556c2e710fe9 0x556c2e70bb0e 0x556c2e69e77a 0x556c2e710e50 0x556c2e70bb0e 0x556c2e69e77a 0x556c2e70d86a 0x556c2e78f7c6 0x556c2e70cee2 0

In [ ]:
!git clone https://github.com/openai/CLIP.git
%cd CLIP

Cloning into 'CLIP'...
remote: Enumerating objects: 64, done.
remote: Total 64 (delta 0), reused 0 (delta 0), pack-reused 64
Unpacking objects: 100% (64/64), done.
/content/CLIP


In [ ]:
import os
import clip
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
# use the following credentials to mount GDrive
# username: clip.fellowshipai@gmail.com
# password: clipteam2021

from google.colab import drive

drive.mount('/content/gdrive')
#drive.flush_and_unmount()

Mounted at /content/gdrive


In [ ]:
!mkdir /content/images/

In [ ]:
!unzip -uq '/content/gdrive/MyDrive/tasteatlas_imgs/imgs.zip' -d "/content/images"

Streaming output truncated to the last 5000 lines.
Bruh/co╠¢m_lam112.jpg:  mismatching "local" filename (Bruh/coтХа┬вm_lam112.jpg),
         continuing with "central" filename version
Bruh/co╠¢m_lam113.jpg:  mismatching "local" filename (Bruh/coтХа┬вm_lam113.jpg),
         continuing with "central" filename version
Bruh/co╠¢m_lam114.jpg:  mismatching "local" filename (Bruh/coтХа┬вm_lam114.jpg),
         continuing with "central" filename version
Bruh/co╠¢m_lam115.jpg:  mismatching "local" filename (Bruh/coтХа┬вm_lam115.jpg),
         continuing with "central" filename version
Bruh/co╠¢m_lam116.jpg:  mismatching "local" filename (Bruh/coтХа┬вm_lam116.jpg),
         continuing with "central" filename version
Bruh/co╠¢m_lam117.jpg:  mismatching "local" filename (Bruh/coтХа┬вm_lam117.jpg),
         continuing with "central" filename version
Bruh/co╠¢m_lam118.jpg:  mismatching "local" filename (Bruh/coтХа┬вm_lam118.jpg),
         continuing with "central" filename version
Bruh/co╠¢m_lam119.

In [ ]:
# !rm -r /content/images/

In [ ]:
# define dictionary to be used for class/label-name correction

classes = {'Fasolakia': 'Fasolakia',
 'apple_pie': 'apple_pie',
 'ayam_goreng': 'ayam_goreng',
 'ayam_penyet': 'ayam_penyet',
 'babi_panggang': 'babi_panggang',
 'baguette': 'baguette',
 'bakaliaros': 'bakaliaros',
 'baked_ziti': 'baked_ziti',
 'bal_kaymak': 'bal_kaymak',
 'bamies': 'bamies',
 'banana_bread': 'banana_bread',
 'batagor': 'batagor',
 'ba╠ünh bo╠Ç': 'banh bo',
 'ba╠ünh_chu╠¢ng': 'banh_chung',
 'ba╠ünh_flan': 'banh_flan',
 'ba╠ünh_gai': 'banh_gai',
 'ba╠ünh_te╠ë': 'banh_tet',
 'ba╠ünh_trung_thu': 'banh_trung_thu',
 'bebek_goreng': 'bebek_goreng',
 'beggars_chicken': 'beggars_chicken',
 'biber_dolmas─▒': 'biber_dolmasi',
 'bindaetteok': 'bindaetteok',
 'bougatsa': 'bougatsa',
 'boyoz': 'boyoz',
 'bread_pudding': 'bread_pudding',
 'bria╠üm': 'briam',
 'brioche': 'brioche',
 'brownies': 'brownies',
 'bruschetta': 'bruschetta',
 'budae_jjigae': 'budae_jjigae',
 'bungeoppang': 'bungeoppang',
 'cag╠å_kebab─▒': 'cag_kebabi',
 'canele': 'canele',
 'cannoli': 'cannoli',
 'cassata': 'cassata',
 'char_siu': 'char_siu',
 'chashu': 'chashu',
 'chateaubriand': 'chateaubriand',
 'chicken_nuggets': 'chicken_nuggets',
 'chicken_parmigiana': 'chicken_parmigiana',
 'chouquette': 'chouquette',
 'ciabatta': 'ciabatta',
 'ciambella': 'ciambella',
 'clafoutis': 'clafoutis',
 'corn_on_the_cob': 'corn_on_the_cob',
 'cornbread': 'cornbread',
 'coto_makassar': 'coto_makassar',
 'co╠¢m_lam': 'com_lam',
 'co╠êp_s╠ºis╠º': 'cop_sis',
 'croissant': 'croissant',
 'croque_monsieur': 'croque_monsieur',
 'crostata': 'crostata',
 'cupcake': 'cupcake',
 'dacquoise': 'dacquoise',
 'dadar_gulung': 'dadar_gulung',
 'dak_galbi': 'dak_galbi',
 'dakbokkeumtang': 'dakbokkeumtang',
 'dara_thong': 'dara_thong',
 'domates_c╠ºorbas': 'domates_corbasi',
 'eclair_decorated': 'eclair',
 'egg_tart': 'egg_tart',
 'escargot': 'escargot',
 'etli_ekmek': 'etli_ekmek',
 'falafel': 'falafel',
 'fanouropita': 'fanouropita',
 'farfalle': 'farfalle',
 'fattoush': 'fattoush',
 'fish_fry': 'fish_fry',
 'flamiche': 'flamiche',
 'focaccia': 'focaccia',
 'fried_chicken': 'fried_chicken',
 'fudge': 'fudge',
 'f─▒r─▒n_su╠êtlac╠º': 'firin_sutlac',
 'galaktoboureko': 'galaktoboureko',
 'galatopita': 'galatopita',
 'galette_des_rois': 'galette_des_rois',
 'garides_saganaki': 'garides_saganaki',
 'gaziantep_baklavas─▒': 'gaziantep_baklavas',
 'giouvetsi': 'giouvetsi',
 'githeri': 'githeri',
 'gnocchi': 'gnocchi',
 'gorengan': 'gorengan',
 'graham_cracker': 'graham_cracker',
 'gratin': 'gratin',
 'gratin_dauphinois': 'gratin_dauphinois',
 'grilled_cheese': 'grilled_cheese',
 'grissini': 'grissini',
 'gu╠êllac╠º': 'gullac',
 'hawawshi': 'hawawshi',
 'haydari': 'haydari',
 'hot_chicken': 'hot_chicken',
 'hu╠ênkar_beg╠åendi': 'hunkar_begendi',
 'inego╠êl_ko╠êfte': 'inegol_kofte',
 'izmir_ko╠êfte': 'izmir_kofte',
 'jambalaya': 'jambalaya',
 'kabak_tatl─▒s─▒ from here': 'kabak_tatlisi',
 'kai_ho_bai_toei': 'kai_ho_bai_toei',
 'kalitsounia': 'kalitsounia',
 'karaage': 'karaage',
 'karn─▒yar─▒k': 'karniyarik',
 'karydopita': 'karydopita',
 'kasutera': 'kasutera',
 'keftedakia': 'keftedakia',
 'kemalpas╠ºa': 'kemalpasha',
 'ketupat': 'ketupat',
 'key_lime_pie': 'key_lime_pie',
 'khanom_khrok': 'khanom_khrok',
 'khanom_tan': 'khanom_tan',
 'khobz': 'khobz',
 'kibbeh': 'kibbeh',
 'kleftiko': 'kleftiko',
 'kol_bo╠êreg╠åi': 'kol_boregi',
 'kolokithopita': 'kolokithopita',
 'korokke': 'korokke',
 'kourabiedes': 'kourabiedes',
 'kreatopita': 'kreatopita',
 'kue_putu': 'kue_putu',
 'kumpir': 'kumpir',
 'kuna╠äfah': 'kunafah',
 'lasagna': 'lasagna',
 'lava_cake': 'lava_cake',
 'loubia': 'loubia',
 'mac_and_cheese': 'mac_and_cheese',
 'macarons': 'macarons',
 'madeleines': 'madeleines',
 'maeuntang': 'maeuntang',
 'manakish': 'manakish',
 'matbucha': 'matbucha',
 'meatloaf': 'meatloaf',
 'melktert': 'melktert',
 'menemen': 'menemen',
 'mille_feuilles': 'mille_feuilles',
 'milopita': 'milopita',
 'mooncake': 'mooncake',
 'm─▒s─▒r_ekmeg╠åi': 'misir_ekmegi',
 'nakji_bokkeum': 'nakji_bokkeum',
 'negima_yakitori': 'negima_yakitori',
 'new_york_style_cheesecake': 'new_york_style_cheesecake',
 'new_york_style_pizza': 'new_york_style_pizza',
 'nikuman': 'nikuman',
 'oncom': 'oncom',
 'opor_ayam': 'opor_ayam',
 'pa_thong_ko': 'pa_thong_ko',
 'pain_au_chocolat': 'pain_au_chocolat',
 'pain_aux_raisins': 'pain_aux_raisins',
 'panettone': 'panettone',
 'paris_brest': 'paris_brest',
 'pastiera': 'pastiera',
 'pastilla_pie': 'pastilla_pie',
 'pastitsio': 'pastitsio',
 'patl─▒can_salatas─▒': 'patlican_salatasi',
 'paximadi': 'paximadi',
 'pecan_pie': 'pecan_pie',
 'peking_duck': 'peking_duck',
 'penne': 'penne',
 'pernil': 'pernil',
 'pestil': 'pestil',
 'pisang_goreng': 'pisang_goreng',
 'pita_bread': 'pita_bread',
 'pizza': 'pizza',
 'pizzette': 'pizzette',
 'porchetta': 'porchetta',
 'portokalopita': 'portokalopita',
 'pot_pie': 'pot_pie',
 'profiterole': 'profiterole',
 'pulled_pork': 'pulled_pork',
 'qua╠é╠ëy': 'quay',
 'quiche': 'quiche',
 'quiche_au_fromage_opt': 'quiche_au_fromage',
 'quiche_lorraine_opt': 'quiche_lorraine',
 'ramazan_pidesi': 'ramazan_pidesi',
 'ratatouille': 'ratatouille',
 'ravioli': 'ravioli',
 'red_velvet_cake': 'red_velvet_cake',
 'rendang_ayam': 'rendang_ayam',
 'revani': 'revani',
 'revithia': 'revithia',
 'risoles': 'risoles',
 'roast_quab': 'roast_quab',
 'sable': 'sable',
 'sekerpare': 'sekerpare',
 'senbei': 'senbei',
 'serabi': 'serabi',
 'sfincione': 'sfincione',
 'shaobing': 'shaobing',
 'sicilian_pizza': 'sicilian_pizza',
 'sigara bo╠êreg╠åi': 'sigara boregi',
 'siu_yuk': 'siu_yuk',
 'souffle': 'souffle',
 'souffle_au_chocolat': 'souffle_au_chocolat',
 'soutzoukakia_smyrneika': 'soutzoukakia_smyrneika',
 'soy_sauce_chicken': 'soy_sauce_chicken',
 'so╠êbiyet': 'sobiyet',
 'stromboli': 'stromboli',
 'supangle': 'supangle',
 'surf_and_turf': 'surf_and_turf',
 'taiyaki': 'taiyaki',
 'tarhana_c╠ºorbas─▒': 'tarhana_corbasi',
 'tarte_flambee': 'tarte_flambee',
 'temaki': 'temaki',
 'tempeh': 'tempeh',
 'teriyaki': 'teriyaki',
 'thot_man_kung': 'thot_man_kung',
 'tie╠é╠üt_canh': 'tiet_canh',
 'tiropita': 'tiropita',
 'tod_man pla': 'tod_man pla',
 'tombik': 'tombik',
 'tori_katsu': 'tori_katsu',
 'tortellini': 'tortellini',
 'tsoureki': 'tsoureki',
 'tteokbokki': 'tteokbokki',
 'tteokgalbi': 'tteokgalbi',
 'vasilopita': 'vasilopita',
 'wingko': 'wingko',
 'xianbing': 'xianbing',
 'xo╠éi_ga╠é╠üc': 'xoi_gac',
 'yakitori': 'yakitori',
 'yaksik': 'yaksik',
 'yemista': 'yemista',
 'yumurtal─▒_pide': 'yumurtali_pide',
 'zaalouk': 'zaalouk'}

# Zero-shot on Tasteatlas (27150 images-289 classes) ~ 35.9%

In [ ]:
# Load the CLIP model
#device = "cuda" if torch.cuda.is_available() else "cpu"
device = 'cpu'
model, preprocess = clip.load('ViT-B/32', device)

input_resolution = model.input_resolution.item()
context_length = model.context_length.item()
vocab_size = model.vocab_size.item()

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

100%|████████████████████████████████████████| 354M/354M [00:03<00:00, 116MiB/s]


Model parameters: 151,277,313
Input resolution: 224
Context length: 77
Vocab size: 49408


In [ ]:
# get images and labels

import os
from tqdm import tqdm

images = []
labels_dist = [] # we're expecting to get distorted label names (character decoding issues)
path = '/content/images/Bruh/'


for f in os.listdir(path):
  if(f!='.DS_Store'):
    image = os.path.join(path, f)
    images.append(image)
    f = f.split('.')[-2]
    stemmed = ''.join([i for i in f if not i.isdigit()])
    labels_dist.append(stemmed)

In [ ]:
len(images)

27150

In [ ]:
len(images),len(labels_dist)

(27150, 27150)

In [ ]:
# check unique classes count

unique = len(set(labels_dist)) # check unique classes
print(f"Unique classes: {unique}")

Unique classes: 289


In [ ]:
# replace distorted label names with correct ones (or at least the ones available so far)

labels = []

for i in labels_dist:
  if i in classes:
    labels.append(classes[i]) # use the pre-defined dictionary
  else:
    labels.append(i)

In [ ]:
len(set(labels)) # re-confirm unique classes count

289

In [ ]:
food_types = list(set(labels)) # create food_types variable

len(food_types)

# food_to_number = {}
# for i, food in enumerate(food_types):
#   food_to_number[food] = i

# number_to_food = {v: k for k, v in food_to_number.items()}

289

In [ ]:
df = pd.DataFrame({'image': images,
                   'label': labels})

In [ ]:
df.head()

,image,label
0,/content/images/Bruh/karydopita86.jpg,karydopita
1,/content/images/Bruh/porchetta55.jpg,porchetta
2,/content/images/Bruh/pita_bread46.jpg,pita_bread
3,/content/images/Bruh/locro_de_zapallo100.jpg,locro_de_zapallo
4,/content/images/Bruh/cupcake28.jpg,cupcake


In [ ]:
sorted_food = sorted(food_types)

In [ ]:
dictionary_key_value = {v:k for k,v in enumerate(sorted_food)}
dictionary_key_value

{'Fasolakia': 0,
 'aji_de_gallina': 1,
 'ajoblanco': 2,
 'albóndigas_en_salsa': 3,
 'almendrados': 4,
 'apple_pie': 5,
 'arroz_con_leche': 6,
 'ayam_goreng': 7,
 'ayam_penyet': 8,
 'babi_panggang': 9,
 'baguette': 10,
 'bakaliaros': 11,
 'baked_ziti': 12,
 'bal_kaymak': 13,
 'bamies': 14,
 'banana_bread': 15,
 'banh bo': 16,
 'banh_chung': 17,
 'banh_flan': 18,
 'banh_gai': 19,
 'banh_tet': 20,
 'banh_trung_thu': 21,
 'batagor': 22,
 'bebek_goreng': 23,
 'beggars_chicken': 24,
 'biber_dolmasi': 25,
 'bindaetteok': 26,
 'birria': 27,
 'blini': 28,
 'bolillo': 29,
 'bougatsa': 30,
 'boyoz': 31,
 'bread_pudding': 32,
 'briam': 33,
 'brioche': 34,
 'brownies': 35,
 'bruschetta': 36,
 'budae_jjigae': 37,
 'bungeoppang': 38,
 'buñuelo': 39,
 'cabrito': 40,
 'cachopo': 41,
 'cag_kebabi': 42,
 'caldo_de_res': 43,
 'canele ()': 44,
 'cannoli': 45,
 'carapulcra': 46,
 'carnitas': 47,
 'cassata': 48,
 'char_siu': 49,
 'chashu': 50,
 'chateaubriand ()': 51,
 'chicken_nuggets': 52,
 'chicken_parm

In [ ]:
list_of_targets = [dictionary_key_value[label] for label in labels]

In [ ]:
df['target'] = list_of_targets

In [ ]:
df.head()

,image,label,target
0,/content/images/Bruh/karydopita86.jpg,karydopita,128
1,/content/images/Bruh/porchetta55.jpg,porchetta,208
2,/content/images/Bruh/pita_bread46.jpg,pita_bread,204
3,/content/images/Bruh/locro_de_zapallo100.jpg,locro_de_zapallo,150
4,/content/images/Bruh/cupcake28.jpg,cupcake,73


In [ ]:
# https://medium.com/analytics-vidhya/dealing-with-pytorch-custom-datasets-64b6c40fe581
class CustomDatasetFromImages(torch.utils.data.Dataset):
    def __init__(self, dataframe, transform):
        """
        Args:
            csv_path (string): path to csv file
            img_path (string): path to the folder where images are
            transform: pytorch transforms for transforms and tensor conversion
        """
        self.data_info = dataframe
        self.image_arr = np.asarray(self.data_info.iloc[:, 0])
        self.label_arr = np.asarray(self.data_info.iloc[:, 2])
        self.data_len = len(self.data_info.index)
        self.transforms = transform

    def __getitem__(self, index):
        single_image_name = self.image_arr[index]
        img_as_img = Image.open(single_image_name).convert("RGB")
        tensor_image = self.transforms(img_as_img)
        single_image_label = self.label_arr[index]

        return (tensor_image, single_image_label)

    def __len__(self):
        return self.data_len

In [ ]:
import numpy as np
import torch
import clip
from PIL import Image

if torch.cuda.is_available():
  device = "cuda"
else:
  device = "cpu"

model, transform = clip.load("ViT-B/32", device=device) #loading the clip checkpoints
input_resolution = model.input_resolution.item()
context_length = model.context_length.item()
vocab_size = model.vocab_size.item()

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

Model parameters: 151,277,313
Input resolution: 224
Context length: 77
Vocab size: 49408


In [ ]:
from torch.utils.data import DataLoader
import torch
import clip
from PIL import Image
from tqdm import tqdm

if torch.cuda.is_available():
  device = "cuda"
else:
  device = "cpu"

model, preprocess = clip.load("ViT-B/32", device=device) #loading the clip checkpoints
test_dataset = CustomDatasetFromImages(df, transform=preprocess)

In [ ]:
torch.manual_seed(42)

def get_features(dataset):
    all_features = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(DataLoader(dataset, batch_size=100)):
            features = model.encode_image(images.to(device))

            all_features.append(features)
            all_labels.append(labels)

    return torch.cat(all_features).cpu().numpy(), torch.cat(all_labels).cpu().numpy()

In [ ]:
test_features, test_labels = get_features(test_dataset)

100%|██████████| 272/272 [38:04<00:00,  8.40s/it]


In [ ]:
id_class_name = {k:v for k,v in enumerate(sorted_food)}
class_id = list(id_class_name.keys())
class_name = list(id_class_name.values())
df_id_class = pd.DataFrame({'class_id':class_id,
                            'class_name':class_name})

In [ ]:
df_id_class

,class_id,class_name
0,0,Fasolakia
1,1,aji_de_gallina
2,2,ajoblanco
3,3,albóndigas_en_salsa
4,4,almendrados
...,...,...
284,284,yakitori
285,285,yaksik
286,286,yemista
287,287,yumurtali_pide


In [ ]:
text_descriptions = [f"This is a photo of a {label}" for label in df_id_class['class_name']]

In [ ]:
texts = clip.tokenize(text_descriptions).to('cpu') 

In [ ]:
def get_topk(texts, test_features, test_labels, top_k=(1,5)):
  with torch.no_grad():
    text_features = model.encode_text(texts).float()
    image_features = test_features
    image_features = torch.from_numpy(image_features)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    top_probs, top_labels = text_probs.cpu().topk(max(top_k), dim=-1)
    pred = top_labels.t()
    target = torch.from_numpy(test_labels)
    correct = pred.eq(target.view(1, -1).expand_as(pred))
    res = []
    for k in top_k:
      correct_k = correct[:k].reshape(-1).float().sum(0)
      res.append(correct_k.mul_(100.0 / len(test_labels)))
    inter_tensor = torch.stack(res)
    top1 = inter_tensor.data[0].item()
    top5 = inter_tensor.data[1].item()
    print(f'Top 1 accuracy is {top1}'.format(top1))
    print(f'Top 5 accuracy is {top5}'.format(top5))


In [ ]:
get_topk(texts, test_features, test_labels, top_k=(1,5))

Top 1 accuracy is 34.1915283203125
Top 5 accuracy is 50.3167610168457


In [ ]:
get_topk(texts, test_features, test_labels, top_k=(1,2))

Top 1 accuracy is 34.1915283203125
Top 5 accuracy is 42.084716796875


In [ ]:
get_topk(texts, test_features, test_labels, top_k=(1,3))

Top 1 accuracy is 34.1915283203125
Top 5 accuracy is 46.022098541259766


In [ ]:
test_labels

array([ 64, 150, 269, ..., 227, 113,  67])

In [ ]:
from sklearn.metrics import confusion_matrix
# Confusion matrix
conf_mat=confusion_matrix(test_labels, predictions)
print(conf_mat)

NameError: ignored

In [ ]:
# %load_ext google.colab.data_table
pd.set_option('display.max_rows', None)
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

plt.figure(figsize=(30,20))

class_names = list(df_id_class['class_name'])
df_cm = pd.DataFrame(conf_mat, index=class_names, columns=class_names).astype(int)
df_cm

NameError: ignored

<Figure size 2160x1440 with 0 Axes>

In [ ]:
# Per-class accuracy
class_accuracy=100*conf_mat.diagonal()/conf_mat.sum(1)
print(class_accuracy)

In [ ]:
df_acc = pd.DataFrame(class_accuracy, index=class_names, columns=['Accuracy_Score']).astype(float)
df_acc.sort_values(by ='Accuracy_Score')

,Accuracy_Score
Fasolakia,0.000000
kabak_tatl─▒s─▒,0.000000
kai_ho_bai_toei,0.000000
kalitsounia,0.000000
karydopita,0.000000
kasutera,0.000000
keftedakia,0.000000
hunkar_begendi,0.000000
kemalpasha,0.000000
khanom_tan,0.000000


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(test_features, test_labels, test_size=0.33, 
                                                    random_state=42, stratify=test_labels)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

#Evaluate L1 regularization strengths for reducing features in final model 
C = [12,13,14] # As C decreases, more coefficients go to zero

for c in C:
    clf = LogisticRegression(penalty='l2', C=c, solver='liblinear', class_weight="balanced")
    # using data before splitting (X_scaled) and (y)
    scores = cross_val_score(clf, X_train, y_train, cv=5, scoring="accuracy")  #<-- add this
    print(scores)  #<-- add this

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


[0.76030786 0.75700935 0.76140737 0.74628917 0.76058274]


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


In [ ]:
from sklearn.linear_model import LogisticRegression
# Perform logistic regression
# *Note that the C value should be determined via a hyperparameter sweep using a validation split.
classifier = LogisticRegression(random_state=0, C=12, verbose=1, solver='liblinear', penalty='l2')
classifier.fit(X_train, y_train)

[LibLinear]

LogisticRegression(C=12, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='liblinear', tol=0.0001, verbose=1,
                   warm_start=False)

In [ ]:
def get_topk(classifier, X_test, y_test, top_k = (1, 5)):
  predictions = classifier.predict(X_test)
  probs = classifier.predict_proba(X_test)
  _max = max(top_k)
  best = np.argsort(probs, axis=1)[:,-(_max):]
  pred = torch.from_numpy(np.transpose(best))
  target = torch.from_numpy(y_test)
  correct = pred.eq(target.view(1, -1).expand_as(pred))
  res = []
  for k in top_k:
    correct_k = correct[:k].reshape(-1).float().sum(0)
    res.append(correct_k.mul_(100.0 / len(y_test)))
  inter_tensor = torch.stack(res)
  top5 = inter_tensor.data[1].item()
  top1 = np.mean((y_test == predictions))*100
  print(f'Top 1 accuracy is {top1}'.format(top1))
  print(f'Top {_max} accuracy is {top5}'.format(_max, top5))

In [ ]:
get_topk(classifier, X_test, y_test, top_k = (1, 5))

Top 1 accuracy is 76.171875
Top 5 accuracy is 93.87277221679688


In [ ]:
get_topk(classifier, X_test, y_test, top_k = (1,2))

Top 1 accuracy is 76.41741071428572
Top 2 accuracy is 86.88616180419922


In [ ]:
get_topk(classifier, X_test, y_test, top_k = (1, 3))

Top 1 accuracy is 76.41741071428572
Top 3 accuracy is 90.6808090209961


In [ ]:
all_features = test_features[0]

In [ ]:
probs = classifier.predict_proba(all_features.reshape(1,-1))
probs

array([[7.50238274e-06, 3.38022325e-05, 9.24722427e-06, 3.32232126e-06,
        2.75991642e-04, 9.53567941e-05, 2.31755916e-04, 1.43433921e-04,
        5.26543847e-05, 1.96328039e-03, 8.09248887e-05, 1.17327025e-05,
        3.56243453e-05, 8.42468994e-05, 5.36409004e-05, 3.40650411e-02,
        2.15558662e-04, 1.16444725e-03, 5.47919816e-05, 1.49104533e-03,
        7.02632843e-05, 3.75970926e-04, 1.86185281e-05, 8.40803812e-05,
        1.51774062e-03, 8.94357160e-06, 7.88674977e-05, 1.16972139e-04,
        1.06474253e-05, 3.54992776e-05, 1.87704264e-04, 1.92737660e-05,
        5.79852618e-04, 1.00240189e-04, 7.23316670e-05, 3.09320404e-01,
        2.94057331e-05, 1.53524332e-05, 1.43189648e-04, 2.36531465e-05,
        2.99531061e-05, 2.96341338e-04, 6.09463161e-04, 7.31256041e-06,
        6.32420835e-03, 1.50606703e-04, 9.83574798e-05, 1.34883131e-04,
        4.07262862e-04, 5.35278135e-04, 6.05788637e-04, 6.35346073e-04,
        5.18769935e-05, 1.61088334e-06, 5.11850827e-04, 2.221661

In [ ]:
top_classes = np.argsort(probs, axis=1)[:,-5:]
top_classes

array([[ 15, 285, 198, 128,  35]])

In [ ]:
test1 = list(np.squeeze(top_classes))
test1 

[15, 285, 198, 128, 35]

In [ ]:
proba = np.squeeze(probs)[np.squeeze(top_classes)]
proba

array([0.03406504, 0.09055188, 0.14915427, 0.23911588, 0.3093204 ])

In [ ]:
food_classes = list(df_id_class['class_name'])

In [ ]:
predicted_class = [food_classes[i] for i in np.squeeze(top_classes)]
predicted_class

['banana_bread', 'yaksik', 'pestil', 'karydopita', 'brownies']

In [ ]:
prob = [prob for prob in proba]
prob

[0.03406504109337038,
 0.09055187855715245,
 0.14915427057092728,
 0.23911587558245212,
 0.30932040378430464]

In [ ]:
import joblib
model = joblib.load('/content/gdrive/MyDrive/linear_probe_288.sav')
probs = model.predict_proba(all_features.reshape(1,-1))
probs

array([[1.50225594e-05, 3.18077717e-05, 9.10006194e-06, 4.48933188e-06,
        1.31527997e-04, 6.55357384e-05, 1.49753965e-04, 1.28125553e-04,
        1.18067237e-04, 1.76672871e-03, 7.67374551e-05, 1.35488348e-05,
        1.76175067e-05, 6.70511298e-05, 5.15356668e-05, 2.58942660e-02,
        1.98318827e-04, 6.98119935e-04, 7.60583189e-05, 2.11453846e-03,
        2.88272744e-05, 1.56585772e-04, 1.13371321e-05, 1.17791064e-04,
        7.78322148e-04, 1.08970483e-05, 6.44475649e-05, 1.21751722e-04,
        1.80622962e-05, 3.98920813e-05, 2.74354052e-04, 1.98479643e-05,
        2.02406705e-03, 1.64122244e-04, 7.75451516e-05, 1.89557250e-01,
        1.39624904e-05, 1.43766141e-05, 1.22854444e-04, 2.96517794e-05,
        2.97528502e-05, 1.16928871e-04, 1.95846615e-04, 8.35814256e-06,
        4.44089357e-03, 1.27344551e-04, 1.99412776e-04, 3.74899855e-04,
        7.25111129e-04, 4.11732214e-04, 3.17569211e-04, 7.09696558e-04,
        8.80616992e-05, 4.15954129e-06, 5.02546326e-04, 1.659480

In [ ]:
top_classes = np.argsort(probs, axis=1)[:,-5:]
top_classes

array([[ 15, 285, 198,  35, 128]])

In [ ]:
test1 = list(np.squeeze(top_classes))
test1 

[15, 285, 198, 35, 128]

In [ ]:
proba = np.squeeze(probs)[np.squeeze(top_classes)]
proba

array([0.02589427, 0.04477508, 0.10700405, 0.18955725, 0.46482466])

In [ ]:
import pickle
model = pickle.load(open('/content/gdrive/MyDrive/linear_probe_288_pickle.sav', 'rb'))
probs = model.predict_proba(all_features.reshape(1,-1))
probs

array([[1.50225594e-05, 3.18077717e-05, 9.10006194e-06, 4.48933188e-06,
        1.31527997e-04, 6.55357384e-05, 1.49753965e-04, 1.28125553e-04,
        1.18067237e-04, 1.76672871e-03, 7.67374551e-05, 1.35488348e-05,
        1.76175067e-05, 6.70511298e-05, 5.15356668e-05, 2.58942660e-02,
        1.98318827e-04, 6.98119935e-04, 7.60583189e-05, 2.11453846e-03,
        2.88272744e-05, 1.56585772e-04, 1.13371321e-05, 1.17791064e-04,
        7.78322148e-04, 1.08970483e-05, 6.44475649e-05, 1.21751722e-04,
        1.80622962e-05, 3.98920813e-05, 2.74354052e-04, 1.98479643e-05,
        2.02406705e-03, 1.64122244e-04, 7.75451516e-05, 1.89557250e-01,
        1.39624904e-05, 1.43766141e-05, 1.22854444e-04, 2.96517794e-05,
        2.97528502e-05, 1.16928871e-04, 1.95846615e-04, 8.35814256e-06,
        4.44089357e-03, 1.27344551e-04, 1.99412776e-04, 3.74899855e-04,
        7.25111129e-04, 4.11732214e-04, 3.17569211e-04, 7.09696558e-04,
        8.80616992e-05, 4.15954129e-06, 5.02546326e-04, 1.659480

In [ ]:
top_classes = np.argsort(probs, axis=1)[:,-5:]
top_classes

array([[ 15, 285, 198,  35, 128]])

In [ ]:
test1 = list(np.squeeze(top_classes))
test1 

[15, 285, 198, 35, 128]

In [ ]:
proba = np.squeeze(probs)[np.squeeze(top_classes)]
proba

array([0.02589427, 0.04477508, 0.10700405, 0.18955725, 0.46482466])

In [ ]:
import joblib
filename = '/content/gdrive/MyDrive/linear_probe_288.sav'
joblib.dump(classifier, filename)

['/content/gdrive/MyDrive/linear_probe_288.sav']

In [ ]:
food_classes = list(df_id_class['class_name'])

In [ ]:
import pickle

In [ ]:
filename = '/content/gdrive/MyDrive/linear_probe_288_pickle.sav'
pickle.dump(classifier, open(filename, 'wb'))

In [ ]:
# make new JSON file
import json
# make new JSON file
with open('/content/gdrive/MyDrive/food_classes_288.json', 'w') as outfile:
  json.dump(food_classes, outfile)

In [ ]:
    ensembling_templates = [
        "This is a photo of a {}",
    ]

In [ ]:
# Create ensembling of prompts
def crate_weight(food_class_names, ensembling_templates):
  with torch.no_grad():
      zeroshot_weights = []
      # for classname in class_names:
      for classname in food_classes:
        texts = [
              template.format(classname) for template in ensembling_templates
          ]  # format with class
        texts = clip.tokenize(texts).to('cpu')  # tokenize
        class_embeddings = model.encode_text(texts)  # embed with text encoder
        class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
        class_embedding = class_embeddings.mean(dim=0)
        class_embedding /= class_embedding.norm()
        zeroshot_weights.append(class_embedding)
  zeroshot_weights = torch.stack(zeroshot_weights, dim=1).to('cpu')
  return zeroshot_weights

In [ ]:
#Change food class names to target class names
zeroshot_weights = crate_weight(food_classes, ensembling_templates)

In [ ]:
zeroshot_weights_for_cpu = zeroshot_weights.cpu().numpy()
np.save('/content/gdrive/MyDrive/numpy_weights_288', zeroshot_weights_for_cpu)